In [ ]:
import os
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report
import torch

In [ ]:
# Define the correct path to your labeled dataset in CoNLL format
filepath = 'C:/Users/hp/Desktop/KAIM/Week 5/labeled_telegram_data_conll.txt'

In [ ]:
# Check if the file exists before proceeding
if not os.path.exists(filepath):
    raise FileNotFoundError(f"The file was not found at the specified path: {filepath}")

In [ ]:
# Function to load the dataset in CoNLL format
def load_conll_format(filepath):
    sentences = []
    labels = []
    with open(filepath, 'r', encoding='utf-8') as f:
        sentence = []
        label = []
        for line in f:
            if line == "\n":  # Sentence boundary
                if sentence:  # Check if sentence is not empty
                    sentences.append(sentence)
                    labels.append(label)
                sentence = []
                label = []
            else:
                token, tag = line.strip().split()
                sentence.append(token)
                label.append(tag)
    return sentences, labels

In [ ]:
# Load the CoNLL dataset
sentences, labels = load_conll_format(filepath)

In [ ]:
# Convert to a pandas DataFrame and then to a Hugging Face Dataset
df = pd.DataFrame({"tokens": sentences, "ner_tags": labels})
dataset = Dataset.from_pandas(df)

In [ ]:
# Load the pre-trained tokenizer and model
model_checkpoint = "xlm-roberta-base"  # You can switch to "bert-tiny-amharic" if available
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# Define label mapping
label_list = ['O', 'B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE']
label_to_id = {label: i for i, label in enumerate(label_list)}

In [ ]:
# Function to tokenize the dataset and align labels with tokens
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True, padding=True)
    labels = []
    for i, label in enumerate(batch['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore subwords and special tokens
            elif word_idx != previous_word_idx:  # Beginning of a new word
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])  # Continuation of the same word
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Tokenize the dataset and align the labels
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
# Set training arguments with matching save_strategy and evaluation_strategy
training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",  # Evaluating at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True  # Load the best model based on evaluation metric at the end
)

In [ ]:
# Load pre-trained model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
# Data collator to pad sequences for token classification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Split dataset into training and validation sets
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [ ]:
# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda p: classification_report(p.label_ids, p.predictions.argmax(-1), output_dict=True)
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model performance on the validation set
trainer.evaluate()

In [ ]:
# Save the fine-tuned model for future use
trainer.save_model("./fine_tuned_ner_model")
tokenizer.save_pretrained("./fine_tuned_ner_model")